In [33]:
from prodock.io.pdb_query import PDBQuery
proc = PDBQuery("5N2F", "Data/testcase", chains=["A", "B"], ligand_code="8HW")
proc.run_all()
print("raw:", proc.cocrystal_ligand_path)
print("processed:", proc.reference_ligand_path)

raw: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/cocrystal/5N2F.sdf
processed: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/reference_ligand/8HW.sdf


1 molecule converted
1 molecule converted


In [34]:
from prodock.preprocess.gridbox import GridBox
from prodock.vis.provis import ProVis
from prodock.vis.provis_gui import ProVisGUI


In [35]:
gb = GridBox().load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
              .from_ligand_pad(pad=4.0, isotropic=True)

print("Box dict for Vina:", gb.vina_dict)
print("Box config snippet:\n", gb.to_vina_lines())


Box dict for Vina: {'center_x': 32.536, 'center_y': 12.991, 'center_z': 133.858, 'size_x': 23.594, 'size_y': 23.594, 'size_z': 23.594}
Box config snippet:
 center_x = 32.536
center_y = 12.991
center_z = 133.858
size_x = 23.594
size_y = 23.594
size_z = 23.594


In [36]:
viz = ProVis(800, 600)
viz.load_receptor("Data/testcase/filtered_protein/5N2F.pdb") \
   .set_receptor_style("cartoon", "white") \
   .load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
   .highlight_ligand(style="stick", color="cyan") \
   .add_gridbox_from(gb, labels=True) \
   .show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

<ProVis models=2, ligands=1>

In [ ]:
gui = ProVisGUI().build().display()


In [38]:
from prodock.preprocess.protein_process import ProteinProcess

pp = ProteinProcess(enable_logging=True)
pp.fix_and_minimize_pdb(
    input_pdb="Data/testcase/filtered_protein/5N2F.pdb",
    output_dir="Data/testcase/filtered_protein/",
    energy_diff=10.0,
    max_minimization_steps=5000,
    pdb_id="5N2F",
    protein_name="5N2F",
    minimize_in_water=False,
    input_ligand='Data/testcase/reference_ligand/8HW.sdf',
    out_fmt="pdbqt",   # default; will produce Data/testcase/filtered_protein/5N2F_min.pdb
)

# print("Artifact:", pp.final_pdb_path)
print("Report:", pp.last_simulation_report)


2025-08-24 15:09:07,183 - INFO - ProteinProcess: fixing Data/testcase/filtered_protein/5N2F.pdb -> Data/testcase/filtered_protein (out_fmt=pdbqt)
2025-08-24 15:09:08,402 - DEBUG - OpenMM platform selected: CPU
2025-08-24 15:09:08,437 - INFO - Minimizing (gas phase)... tol=10.0 maxIter=5000
2025-08-24 15:09:21,815 - DEBUG - Fallback box from protein coords computed.
2025-08-24 15:09:21,816 - DEBUG - Calling mekoo: /homes/biertank/tieu/miniconda3/envs/prodock/bin/mk_prepare_receptor.py --read_pdb Data/testcase/filtered_protein/5N2F.pdb -o Data/testcase/filtered_protein/5N2F --box_center 32.0 8.75 123.75 --box_size 65.25 65.25 65.25 --write_pdbqt Data/testcase/filtered_protein/5N2F.pdbqt
2025-08-24 15:09:26,845 - DEBUG - mekoo rc=0 stdout_len=103 stderr_len=0
2025-08-24 15:09:26,847 - DEBUG - mekoo produced: ['/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.pdbqt', '/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/fi

Report: {'final_artifact': '/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.pdbqt', 'out_fmt': 'pdbqt', 'mekoo_info': {'called': '/homes/biertank/tieu/miniconda3/envs/prodock/bin/mk_prepare_receptor.py --read_pdb Data/testcase/filtered_protein/5N2F.pdb -o Data/testcase/filtered_protein/5N2F --box_center 32.0 8.75 123.75 --box_size 65.25 65.25 65.25 --write_pdbqt Data/testcase/filtered_protein/5N2F.pdbqt', 'rc': 0, 'stdout': '\nFiles written:\nData/testcase/filtered_protein/5N2F.pdbqt <-- static (i.e., rigid) receptor input file\n', 'stderr': '', 'produced': ['/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.pdbqt', '/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.pdbqt']}, 'adt_info': {}, 'minimized_stage': 'gas', 'grid_params': None}


In [39]:
from prodock.io.convert import Converter
conv = Converter().set_input("Data/testcase/reference_ligand/8HW.sdf").set_output("Data/testcase/reference_ligand/8HW.pdbqt").set_mode("ligand").use_meeko().run()
print("output:", conv.output)
print("requested mode:", conv._mode)        # 'ligand'
print("preferred backend:", conv._prefer)  # 'meeko' (because you called use_meeko())


output: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/reference_ligand/8HW.pdbqt
requested mode: ligand
preferred backend: meeko


In [40]:
from prodock.engine.vina import VinaDock

In [ ]:
vd = (VinaDock(sf_name="vina", cpu=4, seed=42)
      .set_receptor("Data/testcase/filtered_protein/5N2F.pdbqt")
      .define_box(center=(32.5, 13.0, 133.75), size=(22.5, 23.5, 22.5))
)

# single ligand
vd.set_ligand("Data/testcase/reference_ligand/8HW.pdbqt").dock(exhaustiveness=8, n_poses=9).write_poses("./Data/testcase/vina/vina_out.pdbqt").write_log("./Data/testcase/vina/log.txt")
print("scores:", vd.scores)
print("best:", vd.get_best())


In [30]:
from prodock.engine.binary import BinaryDock

In [ ]:
bd = (
    BinaryDock("qvina-w")
    .set_receptor("5N2F.pdbqt")
    .set_ligand("8HW.pdbqt")
    .set_out("docked/8HW_docked.pdbqt")
    .set_log("docked/8HW.log")
    .set_box( center=(32.500, 13.0, 133.750), size=(22.500, 23.500, 22.500))
    # .enable_autobox("Data/testcase/reference_ligand/8HW.sdf", padding=4.0)
    .set_exhaustiveness(2)
    .set_num_modes(9)
    .set_cpu(4)
    .set_seed(42)
    .run()
)
# parse rows
rows = bd.parse_scores_from_log("docked/8HW.log")
print(rows)
# write CSV
bd.scores_to_csv("docked/8HW.log", "docked/8HW_scores.csv")
# get pandas DataFrame (requires pandas)
df = bd.scores_as_dataframe("docked/8HW.log")
print(df)


In [ ]:
df